In [1]:
from operator import mul
import urllib2
import numpy as np
import tensorflow as tf
import os
import cv2
import time


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
from tftools import data
import sys
import os
episode_model_save_location = '/data/robotics/stepped/models'
episode_data_save_location = '/data/robotics/stepped/data'
pyrobo_path = '/home/karthik/code/pyrobo/'
sys.path.append(os.path.join(pyrobo_path, 'network_models'))

In [4]:
import trainer
import ps3_control
sess = tf.Session()

In [5]:
def read_episode():
    try:
        with open(os.path.join(episode_data_save_location, 'episode'), 'r') as f:
            current_episode = int(f.readline().strip())
    except IOError:
        current_episode = 0
    return current_episode

def write_episode(episode):
    with  open(os.path.join(episode_data_save_location, 'episode'), 'w') as f:
        f.write("%d"% (episode))

current_episode = read_episode()

In [6]:
tr = trainer.HomeTrainer(sess, episode_model_save_location,
                         episode_data_save_location, current_episode)

In [7]:
robot_url =  'http://localhost:8889/robot/%d/%d/%d/%d/%d/%d/'
ps3_url = 'http://localhost:8888/get'
controller = ps3_control.PS3_Control(robot_url, ps3_url)

http://localhost:8889/robot/1500/1500/1500/1500/1500/1500/


In [8]:
controller.reset()

http://localhost:8889/robot/1500/1500/1500/1500/1500/1500/


In [9]:
controller.get_ps3_out()

[]

In [10]:
while len(controller.get_ps3_out()) == 0:
    current_episode = read_episode()
    print current_episode
    writer = data.TFDataWriter(episode_data_save_location + "/episode_%d.tfrecords"%current_episode)
    episode_end = False
    try:
        while not episode_end:
            time.sleep(0.1)
            pressed = controller.get_ps3_out()        
            for press in pressed:
                if press == ps3_control.EPISODE_END_BUTTON:
                    print "coming here"
                    episode_end = True
                    break
                if press in ps3_control.button_motor_map or press == ps3_control.USE_MODEL_BUTTON:
                    if press == ps3_control.USE_MODEL_BUTTON:
                        motor, direction = tr.predict(controller.motor_state_array())
                    else:
                        motor, direction = ps3_control.button_motor_map.get(press,[None, None])
                        if motor is None:
                            continue
                        writer.add( tr.get_data_to_save() + 
                                    [['motor', [motor], data.INT_TYP],
                                    ['direction', [direction], data.INT_TYP],
                                    ['motor_state', controller.motor_state_array() , data.FLT_TYP]])
                    controller.update_state(motor, direction)
    finally:
        writer.close()
        controller.reset()
        write_episode(current_episode+1)
        time.sleep(2)
        print current_episode

0
http://localhost:8889/robot/1500/1475/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1450/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1475/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1500/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1525/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1550/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1575/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1600/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1625/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1650/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1675/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1700/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1725/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1750/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1775/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1800/1500/1500/1500/1500/
http://localhost:8889/robot/1500/1825/1500/1500/1500/1

KeyboardInterrupt: 

In [ ]:

current_episode = read_episode()
tr.retrain_model(current_episode, 20000, 100)

In [ ]:
left, right = tr.get_images()

plt.imshow(left)


In [ ]:
def get_categorical(prob_n):
    csprob = np.cumsum(prob_n)
    return (csprob > np.random.rand()).argmax()
get_categorical(np.array([0.3, 0.2, 0.4, 0.1]))